### Classe
#### Uma classe é um modelo ou molde a partir do qual objetos são criados, mas não é um objeto. É um tipo de dado definido pelo usuário.

In [ ]:
class Device:
    name = 'i0'
    value = 1000
    prefix = 'TEST:DEVICE:'

### Objeto
#### Um objeto é um "pacote" de software que possui estado e comportamento. Objetos são instâncias das classes, seu comportamento e estado são definidos pela classe.

In [ ]:
detector = Device()

#### Com o objeto é possível acessar atributos, como name ou prefix

In [ ]:
detector.name

In [ ]:
detector.prefix

### Construtor
#### É possível definir os atributos na inicialização do objeto, utilizando o init

In [ ]:
class Detector:
    def __init__(self, name, prefix):
        self.name = name
        self.prefix = prefix
        

In [ ]:
detector2 = Detector('i1', 'TEST:DETECTOR:')

In [ ]:
detector2.name

In [ ]:
detector2.prefix

### Herança e interfaces

### Considere o seguinte problema: 
Com o que aprendemos até agora podemos implementar um simples sistema de aquisição de devices. Basicamente a classe Beamline tem atributos que representam uma lista de seus dispositivos. É possível representar cada dispositivo por uma classe que implementa seus métodos de aquisição de dados.

A classe Beamline é responsável por: 

Adicionar os devices, listar todos os devices e adquirir cada detector.

In [ ]:
class Photodiode:
    def __init__(self, name):
        self.name = name
    
    def acquire(self):
        print(0.9)  

class IonChamber:
    def __init__(self, name):
        self.name = name
    
    def measure(self):
        print(0.75)

class FluoDetector:
    def __init__(self, name):
        self.name = name
    
    def trigger_and_read(self):
        print(0.0)
        
        

            

In [ ]:
class Beamline:
    def __init__(self, name):
        self.name = name
        self.photodiodes = []
        self.ionchambers = []
        self.fluodetectors = []

    def add_fluodetector(self, fluodetector):
        self.fluodetectors.append(fluodetector)

    def add_photodiode(self, photodiode):
        self.photodiodes.append(photodiode)

    def add_ionchambers(self, ionchambers):
        self.ionchambers.append(ionchambers)
    
    def get_devices(self):
        device_list = []
        device_list.extend(self.photodiodes)
        device_list.extend(self.ionchambers)
        device_list.extend(self.fluodetectors)
        return device_list

    def acquire_all(self):
        list_of_devices = self.get_devices()
        for device in list_of_devices:
            if isinstance(device, FluoDetector):
                device.trigger_and_read()
            if isinstance(device, IonChamber):
                device.measure()
            if isinstance(device, Photodiode):
                device.acquire()

#### Diagrama:

<div align="center">
  <img src="images/case1.png" alt="Diagrama de classes, a classe Beamline depende de Admins, Programmers e Designers" title="Diagrama de classes em UML">
</div>



### Note que a implementação da classe empresa **depende** explicitamente das classes: Photodiode, IonChamber e FluoDetector. Além disso, o fluxo de controle é de Beamline para os dispositivos, quais implicações essa implementação pode causar?

In [ ]:
my_Beamline = Beamline('Quati')

In [ ]:
i0 = Photodiode('I0')
it = IonChamber('I1')
fluo = FluoDetector('Fluo')


In [ ]:
my_Beamline.add_photodiode(i0)
my_Beamline.add_fluodetector(fluo)
my_Beamline.add_ionchambers(it)

In [ ]:
my_Beamline.acquire_all()

In [ ]:
i2 = IonChamber('I2')

In [ ]:
my_Beamline.add_ionchambers(i2)

In [ ]:
my_Beamline.acquire_all()

Aparentemente nenhuma, já que nosso código está funcionando :). Porém, considere que há a necessidade de implementar um novo tipo de device. Para isso será necessário modificar o código de "Beamline", que no nosso caso já está funcionando perfeitamente. Além disso será necessário criar uma nova classe de dispositivo.

Isso ocorre pois nossas dependencias apontam no mesmo sentido do fluxo de controle. É possível inverter as dependências considerando uma classe intermediária:

#### Revertendo as dependencias

In [ ]:
class Device:   
    def read(self):
        raise NotImplementedError('Each device must implement its own method to read.')

In [ ]:
class Photodiode(Device):
    def __init__(self, name):
        self.name = name
    
    def read(self):
        print(0.9*2)

class IonChamber(Device):
    def __init__(self, name):
        self.name = name
        
    def read(self):
        print(0.7*5)

In [ ]:
class MyBeamline:
    def __init__(self, name):
        self.name = name
        self.devices = []
    
    def add_device(self, device):
        self.devices.append(device)

    def get_devices(self):
        return self.devices

    def acquire_all(self):
        for device in self.get_devices():
            device.read()
            

In [ ]:
my_beamline = MyBeamline('Quati')

In [ ]:
i0 = Photodiode('i0')
i2 = IonChamber('i3')

In [ ]:
my_beamline.add_device(i0)
my_beamline.add_device(i2)

In [ ]:
my_beamline.acquire_all()

#### O mesmo vale para caso tenha uma função que utiliza devices, a implementação "não sabe" da existência de dispositivos como FluoDetector, IonChamber ou Photodiode. Apenas acredita que estes implementam a interface device.

In [ ]:
def acquire_devices(devices: Device):
    for device in devices:
        device.read()

In [ ]:
acquire_devices([i0, i2])

#### E se o dispositivo não implementar a interface?

In [ ]:
class WrongDetector(Device):
    def __init__(self, name):
        self.name = name

In [ ]:
wrongdet = WrongDetector('Teste')
acquire_devices([wrongdet])

#### Podemos prever excessões do tipo NotImplementedError no código do cliente (MyBeamline), caso necessário.

Note que a implementação também se tornou muito mais simples, no início a classe Beamline precisava conhecer cada um dos tipos de dispositivos, como Photodiodes e IonChamber agora possuem uma interface o código de Beamline é completamente agnóstico a implementações de detectores. De fato, para um novo programador que está vendo esse código pela primeira vez, vai perceber que há uma interface que deve ser implementada, caso ele precise implementar um novo tipo de detector.

Diagrama:

<div align="center">
  <img src="images/case2.png" alt="Diagrama de classes, a classe Beamline depende de Admins, Programmers e Designers" title="Diagrama de classes em UML">
</div>

#### O que torna isso possível?
#### O **polimorfismo**

<p style="text-align: center">f(o)</p>

<p style="text-align: center">o.f()</p>

 Na expressão 1 uma função "f" sendo chamada sobre um objeto "o". Desse modo, assumimos que há apenas uma função chamada "f". Por outro lado, na expressão 2, um objeto "o" recebe uma chamada de "f". Nesse caso (2), podemos esperar que diferentes objetos aceitem a chamada "f", portanto, não sabemos exatamente qual comportamento específico de "f" está sendo invocado. Assim, o comportamento de "f" depende do tipo de "o", isto é, é polimórfico.

#### No nosso exemplo os objetos "Device" se aproveitam do polimorfismo com o método read(), pois é responsabilidade de cada tipo de device realizar o print da aquisição.

#### Como MyBeamline depende apenas de Device, a classe não se importa quem está implementando o método read, apenas espera que a classe implemente o método read.